In [6]:
from cookbook.interface import PluginInstanceRedisInterface
import os
import sys

import logging

logging.basicConfig()
logger = logging.getLogger(__name__)

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379
redis_password = ''
redis_channel = os.environ.get("REDIS_CHANNEL")

# Increase the recursion limit in order to properly serialize Complexes
recursion_limit = 100000
sys.setrecursionlimit(recursion_limit)

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)


In [2]:
from nanome.api.structure import Complex
from nanome.util import Color
# Load Test Molecules
examples_folder = os.path.join(os.getcwd(), 'examples')
pdb_6nzt = os.path.join(examples_folder, '6nzt.pdb')
pdb_6nzv = os.path.join(examples_folder, '6nzv.pdb')

comp_1 = Complex.io.from_pdb(path=pdb_6nzt)
comp_1.name = "6NZT"
# import ipdb; ipdb.set_trace()
comp_2 = Complex.io.from_pdb(path=pdb_6nzv)
comp_2.name = "6NZV"
plugin_instance.update_structures_deep([comp_1, comp_2])
print(comp_1.index)
assert comp_1.index != -1

Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 8c157b91-e2ce-4b13-ba34-aab9226398d5
-1


AssertionError: 

In [7]:
ws = plugin_instance.request_workspace()
comp_list = ws.complexes

Sending request_workspace Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 359a6a2d-f28f-4536-949d-9e7d3c0673e8


In [ ]:
# Run FPocket, and highlight 
from rmsd_v2.fpocket_client import FPocketClient
import time

comp = next(iter(cmp for cmp in comp_list if cmp.name == '6NZT'))
print(len(list(comp.atoms)))
fpocket_client = FPocketClient()
logger.info(f"Showing pockets for {comp.name}")
pocket_sets = fpocket_client.get_pockets(comp)

start_time = time.time()
current_index = 0

while time.time() - start_time < 30:
    print(f"Highlighting pocket {current_index}")
    for atom in comp.atoms:
        atom.selected = atom in pocket_sets[current_index]
    plugin_instance.update_structures_deep([comp])
    current_index = (current_index + 1) % len(pocket_sets)
# Deselect all atoms
for atom in comp.atoms:
    atom.selected = False
plugin_instance.update_structures_deep([comp])

1613
Highlighting pocket 0
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 1e43a7e3-73ab-4ffc-972d-1cdfb0127362
Highlighting pocket 1
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel a57edecc-5f49-42f1-87df-81e47a2ba5a4
Highlighting pocket 2
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel a01c6619-758a-4b8f-a9a1-0fdcb147d8a9
Highlighting pocket 3
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 4317dad7-158e-4f81-a0ce-476bf74f37f7
Highlighting pocket 4
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel e1a182e6-38e5-4216-8f26-15579cfdee6d
Highlighting pocket 0
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f8